In [ ]:
import pandas as pd
import plotly.graph_objects as go

# Load original data
overview_df = pd.read_csv("../overview.csv")
overview_df['Date'] = pd.to_datetime(overview_df['Date'])

# Load financial events data
events_df = pd.read_csv("amenity_extractions.csv")
events_df['Financial Events Publication Date'] = pd.to_datetime(events_df['Financial Events Publication Date'])

# Rename columns for easier manipulation
events_df.rename(columns={
    'Financial Events Publication Date': 'Date',
    'Company Info Company': 'Company Ticker',
    'Financial Events Polarity': 'Polarity'
}, inplace=True)

# Calculate sentiment score for each date and company
polarity_count = events_df.groupby(['Date', 'Company Ticker', 'Polarity']).size().unstack(fill_value=0)
polarity_count['Sentiment Score'] = (polarity_count.get('pos', 0) - polarity_count.get('neg', 0)) / \
                                    (polarity_count.get('pos', 0) + polarity_count.get('neg', 0) + 1)

polarity_count.reset_index(inplace=True)

# Window size for moving average
window_size = 5

# Create a Plotly figure
fig = go.Figure()

# Plotting original data: moving average sentiment score
for company in overview_df['Company Ticker'].unique():
    company_df = overview_df[overview_df['Company Ticker'] == company].copy()
    company_df.sort_values('Date', inplace=True)
    company_df['MA Sentiment Score'] = company_df['Sentiment Score'].rolling(window=window_size, min_periods=1).mean()
    fig.add_trace(go.Scatter(x=company_df['Date'], y=company_df['MA Sentiment Score'], mode='lines+markers', name=f"{company} MA Sentiment"))

# Plotting new data: moving average of sentiment score from financial events
for company in polarity_count['Company Ticker'].unique():
    company_df = polarity_count[polarity_count['Company Ticker'] == company]
    company_df.sort_values('Date', inplace=True)
    company_df['MA Sentiment Score'] = company_df['Sentiment Score'].rolling(window=window_size, min_periods=1).mean()
    fig.add_trace(go.Scatter(x=company_df['Date'], y=company_df['MA Sentiment Score'], mode='lines', name=f"{company} Events MA Sentiment"))

# Update layout
fig.update_layout(title='Comparison of Company Sentiment Scores Over Time (Smoothed)',
                  xaxis_title='Date',
                  yaxis_title='Sentiment Score',
                  template='plotly_dark')

# Show the plot
fig.show()
